<a href="https://colab.research.google.com/github/sahin-arslan/ML_Final_Odevi/blob/main/MLFinalOdev.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Veri Seti**

Pima kızılderilerine ait 5 yıllık tıbbi veriler içermektedir. Bu veriler ile diyabet bağlanğıçının tahmini konusunda çalışma yapacağız.

İki sınıflı bir veri seti vardır elimizde. Yani sonuçumuz 1 yada 0 şekilindedir. Veriler homojen dağılmamıştır. Bazı eksik verilerin sıfır olarak kabul edildiği varsayılmaktadır.


Veri setimizdeki alanlar:
1.   NOTP : Gebelik sayısı
2.   PGC : Oral glikoz tolerans testinde 2 saat plazma glikoz konsantrasyonu
3.   DBP : Diyastolik kan basıncı (mm Hg)
4.   TST : Triceps deri kıvrım kalınlığı (mm)
5.   INS : 2 saat serum insülini (mu U/ml)
6.   BMI : Vücut kitle indeksi (kg cinsinden ağırlık/(m cinsinden boy)^2).
7.   DPF : Diyabet soyağacı fonksiyonu
8.   Age : Yaş
9.   CV : Sınıf değişkeni(0 veya 1) (Tahmin edilmek istenen değişken)



Not : Veri setine kolon isimleri eklenmiştir.

**1.	Size atanan veri setine tüm algoritmalar için optimizasyon uygulayınız.**


*   Normalizasyon işlemi için Min-Max yöntemini tercih ettik.
*   Bağımlı ve bağımsız değişkenlerimizi ayrımlarını yaprak datasetlerimizi oluşturduk
*   Eğitim ve test setlerimizi ayrıştırdık(%20 test)
*   Model hiperparametlererini belirledik. Burada RandomForestRegressor'den yararlanıyoruz
*   Grid Search kullanarak hiperparemater optimizasyonu uyguluyoruz.
*   En iyi parametrelerimizi belirledikten sonra ekrana yazdırıyoruz.







In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Veri setini yüklüyoruz
df = pd.read_csv('veri-seti.txt', sep="\t")

# Eksik verileri kontrol ediyoruz(Null değer varmı buna bakıyoruz)
print(df.isnull().sum())

# Normalizasyon
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df_scaled = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)

# Özellik ve hedef değişkenleri ayırıyoruz
X = df_scaled.drop('CV', axis=1)
y = df_scaled['CV']

# Veriyi eğitim ve test setlerine ayırıyoruz
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#En İyi Parametreler:

# Model ve Hiperparametreler
model = RandomForestRegressor(random_state=42)
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Grid Search ile Hiperparametre Optimizasyonu yapıyoruz
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)

# En iyi parametreleri ekrana yazdırıyoruz
print(f'\nEn iyi parametre Listesi: {grid_search.best_params_}')

# En iyi model ile tahmin yapıyoruz
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

# Model performansını değerlendiriyoruz
mse = mean_squared_error(y_test, y_pred)
print(f'\nMSE (Tahminlerin gerçek değerlerden ne kadar saptığı:): {mse}')


NOTP    0
PGC     0
DBP     0
TST     0
INS     0
BMI     0
DPF     0
Age     0
CV      0
dtype: int64
Fitting 3 folds for each of 108 candidates, totalling 324 fits

En iyi parametre Listesi: {'max_depth': None, 'min_samples_leaf': 2, 'min_samples_split': 10, 'n_estimators': 50}

MSE (Tahminlerin gerçek değerlerden ne kadar saptığı:): 0.16973497650428213


**Sonuçların yorumlanması**
Paremetre listesini kullanarak en iyi performans için parametrelerimizi aşağıdaki gibi belirledik. Herbir kalemde ilgili parametreye ait açıklamayı aşağıda ekledik.

Parametreler:

1.   max_depth : None (olarak çıktı bu da ağaçımızın büyüme konusunda bir sınır koymayacağımız anlamına geliyor.)
2.   min_samples_leaf : 2 (Bir düğüm de bulunması gereken minimum dal sayısı)
3.   min_samples_split: 10 (bir sonraki düğüme dallanmak için gerekli minumun öğe sayısı)
4.   n_estimators: 50(oluşturulacak ağaç sayısı)






**2. Veri setinizi rastgele olarak %70 eğitim %30 test olacak şekilde ayırınız. Eğitim veri seti için Naive bayes sınıflandırıcısını uygulayınız. Elde ettiğiniz sonucları raporlayınız.**

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, classification_report

# Hedef değişkeni ve bağımsız değişkenleri ayırın
X = df.drop('CV', axis=1)
y = df['CV']

# Veri setini rastgele %70 eğitim %30 test olacak şekilde ayırın
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Naive Bayes sınıflandırıcısını oluşturun ve eğitin
nb_classifier = GaussianNB()
nb_classifier.fit(X_train, y_train)

# Test verileri üzerinde tahmin yapın
y_pred = nb_classifier.predict(X_test)

# Sonuçları raporlayın
accuracy = accuracy_score(y_test, y_pred)
print("Doğruluk:", accuracy)
print("\nSınıflandırma Raporu:\n", classification_report(y_test, y_pred))

Doğruluk: 0.7445887445887446

Sınıflandırma Raporu:
               precision    recall  f1-score   support

           0       0.82      0.79      0.80       151
           1       0.62      0.66      0.64        80

    accuracy                           0.74       231
   macro avg       0.72      0.73      0.72       231
weighted avg       0.75      0.74      0.75       231



**Genel Doğruluk** değerimiz
    **0.74**'dir.
      Diğer bir değişle modelinizin doğru tahmin etme yüzdesi yaklaşık %74. Bu, modelinizin test verileri üzerindeki tahminlerinin %74'ünün doğru olduğunu gösterir.

**Sınıflandırma Raporu Ayrıntıları**


1.   **Sınıf 0 (Negatif Sınıf)**
    
    Sınıf 0 diyabet başlanğıcı olmayan hastaları ifade etmektedir.

  *   **Precision (Kesinlik): 0.82**
    Modelimizin sınıf 0 için yaptığı pozitif tahminlerin %82'si doğru.

  *   **Recall (Duyarlılık): 0.79** Gerçekten 0 olan yadi diyabet başlancığı olmayan hastaların %79'unu doğru olarak tahminliyoruz. Burada %21'lik bir hata durumumuz mevcut.

  *   **F1-Score: 0.80** Diğer iki parametreden ayrı olarak F1 score'u diyabet başlanğıcı olmayan hastaların genel tamin konusundaki performansına ilişkin bilgi veriyor. Sınıf 0 için %80'lık bir başarı yüzdemiz var diyebiliriz.

  
2.   **Sınıf 1 (Pozitif Sınıf)**
  
  Sınıf 1 diyabet başlanğıcı olan hastaları ifade etmektedir.

  *   **Precision (Kesinlik): 0.62** Modelimizin diyabet başlanğıcı olan hastalardaki tahmin yüzdemiz biraz daha düşük olduğu görülmektedir. Diyabet başlanğıcı olan hastaların %62'sini doğru olarak tamin edebiliyoruz. Buradaha hata payımızı biraz yüksek yanı %39 dir.

  *   **Recall (Duyarlılık): 0.66** Gerçekten diyabet başlanğıcı olan hastalar için bu tahmin yüzdemizi ifade etmektedir. Buradaki başarı oranımız %66'ya denk gelmektedir.

  *   **F1-Score: 0.64** Diyabet başlanğıcı hastların tahmin yüzdesinin genel olarak başarısını ifade etmektedir. Buradaki başarı oranımız %64 hata payımız ise %36 dir.



**Genel Durum Değerlendirmesi**


1.   **Accuracy (Doğruluk):0.74** Tüm veri seti üzerindeki genel tahmin başarı yüzdemizi ifade etmektedir. %74 oranında başarılı tahmin yaptığımızı söyleyebiliriz.
2.   **Macro Average (Makro Ortalama):**

  Precision: 0.72

  Recall: 0.73
  
  F1-Score: 0.72

  Makro ortalama, her iki sınıfın performansını eşit ağırlıkla alır ve ortalama performansı gösterir. Diğer bir değişle ortalama başarı yüzdemizi ifade etmektedir. Bu üç parametreyi öncesinde ayrıntılı olarak ifade ettik.


3.   **Weighted Average (Ağırlıklı Ortalama):**

  Precision: 0.75

  Recall: 0.74

  F1-Score: 0.75

  Ağırlıklı ortalama, her sınıfın veri setindeki temsil oranına göre ağırlıklandırılarak hesaplanır. Diğer bir ifade ile veri seti üzerindeki veri dağılımı dikkate alınarak hesaplandığını ifade etmektedir.


 Precision ve recall değerleri arasında belirgin bir fark olduğu görülüyor. Sınıf 1 için precision ve recall değerleri, sınıf 0'a göre daha düşük. Bu, modelin sınıf 1'i sınıf 0'dan daha az iyi tanıdığı anlamına gelir. Yani diyabet başlanğızı olmayan hastaların tahmini konusunda modelimiz daha başarılı.


  Sonuç olarak, modeliniz sınıf 0'ı(diyabet başlanğıcı olmayan hastalar) iyi bir şekilde tanırken, sınıf 1'de(diyabet başlanğıcı olan hastalar) performans düşüklüğü yaşıyor. Bu durum, modelin sınıf dengesizliği veya sınıf 1'e ait örneklerin daha az belirgin özelliklere sahip olmasından kaynaklanabilir. Bu nedenle, veri dengesizliği ile başa çıkmak ve modelin performansını artırmak için verilerin homojen dağılıma sahip olduğuna dikkat etmek gerekir.

 **3.	Veri setinizi rastgele olarak %70 eğitim %30 test olacak şekilde ayırınız. Eğitim veri seti için K-en yakın komşuluk sınıflandırıcısını uygulayınız. En iyi k değerini belirleyerek Elde ettiğiniz sonuçları raporlayınız.**  



In [ ]:
from sklearn.neighbors import KNeighborsClassifier


# Bağımlı ve bağımsız değişlenlerin ayırıyoruz
X = df.drop('CV', axis=1)
y = df['CV']

# Data'yı eğitim ve test seti olarak ayırıyoruz
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# k değerimiz için bir aralık belirliyoruz. k değerimiz 1 ile 20 arasında bir değer alacaktır.
k_values = range(1, 21)

# Her k değeri için doğruluk scorlarını saklamak adına bir liste oluşturuyoruz
accuracy_scores = []

# Her k değeri için KNeighborsClassifier oluşturuyor
# Eğitim ve test veri ile modeli eğitiyoruz
# Test veri setini kullanarak taminle yapıyoruz
# Bu tahminler doğrultusunda dopruluk deperlerini hesaplayarak accuracy_scores listemize ekliyoruz.
for k in k_values:
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train, y_train)
    y_pred = knn.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    accuracy_scores.append(accuracy)

# En yüksek doğruluk değerine sahip k değerini buluyoruz.
best_k = k_values[accuracy_scores.index(max(accuracy_scores))]

print(f"En iyi k değeri: {best_k}")
print(f"En yüksek doğruluk değeri: {max(accuracy_scores)}")

En iyi k değeri: 13
En yüksek doğruluk değeri: 0.7445887445887446


**En İyi k Değeri : 13**

En iyi k değeri 13 olduğunda, modelimizin performansı en yüksek seviyede olduğunu söyleyebiliriz. Bu, KNN algoritmasının, eğitim verilerindeki en yakın 13 komşuyu kullanarak en iyi sonucu verdiği anlamına geliyor.

**En Yüksek Doğruluk: 0.7445887445887446**

Modelinizin doğruluğu yaklaşık %74. Bu, test veri setinde modelin yaptığı tahminlerin %74'ünün doğru olarak tahmin ettiği anlamına gelir.

**Değerlendirme**

%74 doğruluk skoru iyi bir skor olarak nitelendirebiliriz ama bu oranı artırmak mümkün olabilir.


KNN algoritması, yüksek boyutlu veri setlerinde performans kaybı yaşayabilir. Yani çok girdinin bulunduğu veri setlerini ifade etmektedir. Bizim veri setimizde toplam 8 tane girdi parametremiz bulunmaktadır. Bu durumda, özellik seçimi veya boyut indirgeme yöntemleri kullanarak performans artırabiliriz.
Diğer bir değişle sonuçla kolerasyonu yüksek girdi parametrelerin kullanılması daha sağlıklı sonuçlar verecektir.


Sonuç olarak, elde ettiğimiz sonuçlar doğrultusunda KNN modeli ile %74 doğruluk skorunu yakaladir diyebilirzi. En iyi performansı veren k değeri 13 olarak belirleyebildik.

**4.	Veri setinizi rastgele olarak %70 eğitim %30 test olacak şekilde ayırınız. Eğitim veri seti için Multi-Layer Perceptron (MLP) ve Support Vector Machines (SVM) sınıflandırıcılarını uygulayınız. Eğitim ve test adımlarında elde ettiğiniz sonucları raporlayınız.**

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC

# Özelik ve hedef değişkenlerimizi ayırıyoruz
X = df.drop('CV', axis=1)
y = df['CV']

# Eğitim, test veri setlerimizi ayırıyoruz. Test için %30'luk bir kısım ayırıyoruz
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# MLPClassifier kullanılarak bir MLP (Multilayer Perceptron) modeli oluşturuyoruz
# Test ve eğitim veri testleri kullanarak modelimizi eğitiyoruz
# Eğitimiz modelimizi test veri seti ile test ediyoruz. Yani taminleme yapıyoruz.
# Son adımımızda yapılan taminleme sonuçları ile doğruluk skorunu hesaplıyoruz.
mlp = MLPClassifier(hidden_layer_sizes=(100,), activation='relu', solver='adam', random_state=42)
mlp.fit(X_train, y_train)
y_pred_mlp = mlp.predict(X_test)
accuracy_mlp = accuracy_score(y_test, y_pred_mlp)
print(f"MLP Doğruluk Skoru(Eğitim): {mlp.score(X_train, y_train)}")
print(f"MLP Doğruluk Skoru(Test): {accuracy_mlp}")

# SVM kullanılarak bir SVM (Support Vector Machine) modeli oluşturuyoruz
# Test ve eğitim veri testleri kullanarak modelimizi eğitiyoruz
# Eğitimiz modelimizi test veri seti ile test ediyoruz. Yani taminleme yapıyoruz.
# Son adımımızda yapılan taminleme sonuçları ile doğruluk skorunu hesaplıyoruz.
svm = SVC(kernel='rbf', random_state=42)
svm.fit(X_train, y_train)
y_pred_svm = svm.predict(X_test)
accuracy_svm = accuracy_score(y_test, y_pred_svm)
print(f"\nSVM Doğruluk Skoru(Eğitim): {svm.score(X_train, y_train)}")
print(f"SVM Doğruluk Skoru(Test): {accuracy_svm}")

MLP Doğruluk Skoru(Eğitim): 0.7914338919925512
MLP Doğruluk Skoru(Test): 0.7229437229437229

SVM Doğruluk Skoru(Eğitim): 0.7802607076350093
SVM Doğruluk Skoru(Test): 0.7359307359307359


MLPClassifier ve SVM modellerinin performanslarını inceleyerek karşılaştırmasını yapalım.

**MLPClassifier (Multi-Layer Perceptron)**
1.   **Training Accuracy (Eğitim Doğruluğu): 0.7914338919925512**

    Modelimizin eğitim verisi üzerindeki doğruluk skorunu yaklaşık olarak %79'dur. Bu, modelimizin eğitim veri seti üzerinde yaptığı taminlemenin %79 doğru olduğu anlamına gelmektedir.


2.  **Testing Accuracy (Test Doğruluğu): 0.7229437229437229**

    Modelimizin test verisi üzerindeki doğruluk skorunu yaklaşık olarak %72'dur. Bu, modelimizin test veri seti üzerinde yaptığı taminlemenin %72 doğru olduğu anlamına gelmektedir.

**SVM (Support Vector Machine)**


1.   **Training Accuracy (Eğitim Doğruluğu): 0.7802607076350093**

    Modelimizin eğitim verisi üzerindeki doğruluk skorunu yaklaşık olarak %78'dur. Bu, modelimizin eğitim veri seti üzerinde yaptığı taminlemenin %78 doğru olduğu anlamına gelmektedir.

2.   **Testing Accuracy (Test Doğruluğu): 0.7359307359307359**

    Modelimizin test verisi üzerindeki doğruluk skorunu yaklaşık olarak %73'dur. Bu, modelimizin test veri seti üzerinde yaptığı taminlemenin %73 doğru olduğu anlamına gelmektedir
  
**Karşılaştırma ve Değerlendirme**


**Eğitim Doğruluğu**
  *   MLP: %79
  *   SVM: %78

  Eğitim verisi üzerinde, MLPClassifier biraz daha yüksek bir doğruluğa sahip olduğunu görüyoruz. Bu, MLP modelinin eğitim seti üzerindeki tahminleme yeteneğinin daha iyi olduğunu söyleyebiliriz.

**Test Doğruluğu**


*   MLP: %72
*   SVM: %73

  Test verisi üzerinde, SVM modeli MLP modeline göre biraz daha yüksek bir doğruluk değerine sahip. Bu, SVM modelinin daha önce görmediği verileri biraz daha iyi tahminleme yeteneğine sahip olduğunu gösterir.


Sonuç olarak, SVM modeli test verisi üzerinde biraz daha iyi performans gösteriyor. Buna ek olarak her iki modelin de performansının benzer olduğunu görüyoruz. Farklı algoritmalar ve optimizasyonlarla, daha yüksek doğruluk oranlarına ulaşabiliriz.
